In [ ]:
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
from transformers import CLIPModel, AutoModel, CLIPProcessor
from preprocessing import fcgr, protein_to_dna
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
#from CLIP.clip import clip
#from CLIP.clip.simple_tokenizer import SimpleTokenizer as _Tokenizer
from clip import clip
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pathlib import Path

In [ ]:
from clip import clip
from clip.simple_tokenizer import SimpleTokenizer as _Tokenizer

In [ ]:
class image_title_dataset(Dataset):
    def __init__(self, processor, images, labels, max_len):
        # Initialize image paths and corresponding texts
        self.images = images
        self.max_len = max_len
        # Tokenize text using CLIP's tokenizer
        self.labels = labels
        self.processor = processor
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Preprocess image using CLIP's preprocessing function
        labels_short = self.labels[idx:idx + 1].tolist()[:][:self.max_len]

        tokens = clip.tokenize(self.labels[idx:idx + 1].tolist()[:50], context_length = 256,truncate=True)[:1, :self.max_len]

        if tokens.size()[1] < self.max_len:
            tokens = torch.cat([tokens,
                       torch.zeros(size=(1, self.max_len - tokens.size()[1]))], dim=1).type(
                torch.int)
        #attmask = torch.zeros()
        #pixel_values = torch.transforms

        inputs = self.processor(text=labels_short, images=self.images[idx:idx+1], return_tensors="pt", padding=True)
        inputs['input_ids'] = tokens
        if inputs['attention_mask'].size()[1] > self.max_len:
            inputs['attention_mask'] = inputs['attention_mask'][:1, :self.max_len]
        inputs['attention_mask'] = torch.cat([inputs['attention_mask'], torch.zeros(size=(1, len(inputs['input_ids'][0]) - len(inputs['attention_mask'][0])))], dim=1).type(torch.int)

        #inputs['input_ids'] = torch.cat([inputs['input_ids'], torch.Tensor([[0] * (self.max_len - len(inputs['input_ids'][0]))])], dim=1)
        #inputs['attention_mask'] = torch.cat([inputs['attention_mask'], torch.Tensor([[0] * (self.max_len - len(inputs['attention_mask'][0]))])], dim=1)
        return inputs

In [ ]:
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        if p.requires_grad:
            p.grad.data = p.grad.data.float()

In [ ]:
def evaluate_training(model,loader_test, criterion):
    loss_all = []
    with torch.no_grad():
        
        for i, inputs in enumerate(loader_test):
            outputs = model(input_ids=torch.Tensor(inputs['input_ids']).type(torch.int),attention_mask=torch.Tensor(inputs['attention_mask']).squeeze(1), pixel_values=torch.Tensor(inputs['pixel_values']).squeeze(1))
            
            logits_i = outputs.logits_per_image
            logits_t = outputs.logits_per_text
            #probs = logits.softmax(dim=1)

            labels = torch.arange(0, logits_i.shape[0])
            loss_i = criterion(logits_i, labels)
            loss_t = criterion(logits_t, labels)

            loss = (loss_i + loss_t)/2
            loss_all.append(loss.item())

    return np.mean(loss_all)
        
            

In [ ]:
def train(model, processor, images, labels, epochs=1):
    #tokenizer = get_or_build_tokenizer(labels)
    criterion = nn.CrossEntropyLoss()
    #for img in images:

    #labeltokens = clip.tokenize(labels.tolist(), 128)

    #labeltokens = np.array([np.array(tokenizer.encode(label).ids) for label in labels])

    #inputs = processor(text=labeltokens, images=images, return_tensors='pt', padding=True)
    #inputs = processor(text=torch.Tensor(labeltokens[:1]), images=torch.permute(images[:1], dims=(0,2,3,1)) )
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = Adam(params, lr=1e-5, weight_decay=0.0001)
    
    n=len(labels)
    assert n == images.shape[0]
    
    images_train = images[:int(n*0.9)]
    images_test = images[int(n*0.9):]
    labels_train = labels[:int(n*0.9)]
    labels_test = labels[int(n*0.9):]

    
    dataset_train = image_title_dataset(processor, images_train, labels_train, 77) #instead of 77
    loader_train = DataLoader(dataset_train, batch_size=128, shuffle=True)

    dataset_test = image_title_dataset(processor, images_test, labels_test, 77) #instead of 77
    loader_test = DataLoader(dataset_test, batch_size=128, shuffle=True)


    for epoch in range(epochs):
        loss_epoch = []
        print(f"Epoch {epoch+1}/{epochs}--------------------------------")
        for i, inputs in enumerate(loader_train):
            
        #inputs = processor(text=labels[:].tolist(), images=images[:], return_tensors="pt", padding=True)

            optimizer.zero_grad()
            outputs = model(input_ids=torch.Tensor(inputs['input_ids']).type(torch.int),
                attention_mask=torch.Tensor(inputs['attention_mask']).squeeze(1), pixel_values=torch.Tensor(inputs['pixel_values']).squeeze(1))

            logits_i = outputs.logits_per_image
            logits_t = outputs.logits_per_text
            #probs = logits.softmax(dim=1)

            labels = torch.arange(0, logits_i.shape[0])
            loss_i = criterion(logits_i, labels)
            loss_t = criterion(logits_t, labels)

            loss = (loss_i + loss_t)/2
            loss.backward()

            if device == 'cpu':
                optimizer.step()
            else:
                convert_models_to_fp32(model)
                optimizer.step()
                clip.model.convert_weights(model)

            print(f"{loss.item()}... batch {i+1}/{len(loader_train)}")
            loss_epoch.append(loss.item())
    
            if i%10==0:
                loss_epoch_test = evaluate_training(model,loader_test,criterion)
                print(f"Current eval loss after batch {i+1}/{len(loader_train)} epoch={epoch} = {loss_epoch_test}")

        loss_epoch = np.mean(loss_epoch)
        print(f"Loss epoch {epoch} = {loss_epoch}")
        
        loss_epoch_test = evaluate_training(model,loader_test,criterion)
        print(f"Final eval loss epoch={epoch} = {loss_epoch_test}")



In [ ]:
#model, processor = clip.load("ViT-B/32",device=device,jit=True) #Must set jit=False for training
"""
    model : torch.nn.Module
        The CLIP model


    preprocess : Callable[[PIL.Image], torch.Tensor]
        A torchvision transform that converts a PIL image into a tensor that the returned model can take as its input
"""

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
for k in model.state_dict().keys(): print(k)

In [ ]:
nsamples = 10000
labelcol = "Protein names"
inputcol = "Sequence"
labels = pd.read_csv(r"labels.csv")[labelcol][:nsamples]
inputs = pd.read_csv(r"sequences.csv")[inputcol][:nsamples]


In [ ]:
images = np.array([fcgr(seq, k=7) for seq in inputs])
images = np.array([img/np.sum(img) for img in images])
images = torch.Tensor(images).unsqueeze(1).repeat(1, 3, 1, 1)

In [ ]:
train(model, processor, images, labels, epochs=1)

In [ ]:
import os
model_path = os.getcwd()

from datetime import datetime
timestamp = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")


model_name = f"model_clop_({timestamp}).ckpt"
torch.save(model.state_dict(), os.path.join(model_path, model_name) )

In [ ]:
x = torch.load(os.path.join(model_path, "model_clop_(2023_12_01-12_57_49).ckpt"))

# ****************************************************

In [ ]:
data = image_title_dataset(processor, images[:1000], labels[:1000], max_len=77)
loader = DataLoader(data, batch_size=1000, shuffle=True)
len(loader)
inputs = list(loader)[0]

In [ ]:
outputs = model(input_ids=torch.Tensor(inputs['input_ids']).type(torch.int),
                attention_mask=torch.Tensor(inputs['attention_mask']).squeeze(1), 
                pixel_values=torch.Tensor(inputs['pixel_values']).squeeze(1))

In [ ]:
labels_embeddings = outputs.text_embeds
images_embeddings = outputs.image_embeds

In [ ]:
seqs =["ACGTCGTAGCTAGCTAGCTAGCTAGCTAGTCAGCTA"]
imgs = np.array([fcgr(seq, k=7) for seq in seqs])
imgs = np.array([img/np.sum(img) for img in imgs])
imgs = torch.Tensor(imgs).unsqueeze(1).repeat(1, 3, 1, 1)
lbls = pd.Series(["homo sapiens"])

data = image_title_dataset(processor, imgs, lbls, max_len=77)
loader = DataLoader(data, batch_size=1, shuffle=True)
outputs = model(input_ids=torch.Tensor(inputs['input_ids']).type(torch.int),
                attention_mask=torch.Tensor(inputs['attention_mask']).squeeze(1), 
                pixel_values=torch.Tensor(inputs['pixel_values']).squeeze(1))

lbl_embedding = outputs.text_embeds
img_embedding = outputs.image_embeds

In [ ]:
labels_embeddings.shape
lbl_embedding.shape

In [ ]:
torch.multiply(lbl_embedding, labels_embeddings).shape

In [ ]:

image_encoded = model.vision_model.encoder(inputs['pixel_values'])
image_encoded